<a href="https://colab.research.google.com/github/Mu7annad0/Named-Intity-Rocegnition/blob/main/Ner_XLM_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade datasets --progress-bar off
!pip install --upgrade transformers --progress-bar off
!pip install --upgrade accelerate --progress-bar off
!pip install --upgrade peft --progress-bar off
!pip install --upgrade bitsandbytes --progress-bar off
!pip install --upgrade trl --progress-bar off
!pip install seqeval --progress-bar off

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import DataCollatorForTokenClassification
from transformers import Trainer
from huggingface_hub import notebook_login
from collections import defaultdict, Counter
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score, accuracy_score
import pandas as pd
import torch.nn as nn
import numpy as np
import torch

In [ ]:
dataset = load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
element = dataset["train"][0]
for key, value in element.items():
  print(f"{key}: {value}")

id: 0
tokens: ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
pos_tags: [22, 42, 16, 21, 35, 37, 16, 21, 7]
chunk_tags: [11, 21, 11, 12, 21, 22, 11, 12, 0]
ner_tags: [3, 0, 7, 0, 0, 0, 7, 0, 0]


In [ ]:
# features names
for key, value in dataset["train"].features.items():
  print(f"{key}: {value}")

id: Value(dtype='string', id=None)
tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
pos_tags: Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)
chunk_tags: Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)


In [ ]:
tags = dataset["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)

In [ ]:
def tag_names(batch):
  return {"ner_tagss": [tags.int2str(i) for i in batch["ner_tags"]]}

tag_names(element)

{'ner_tagss': ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']}

In [ ]:
example = tag_names(dataset["train"][10])
pd.DataFrame([dataset["train"][10]["tokens"], example["ner_tagss"]], ["Tokens","Taga"])

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
Tokens,Spanish,Farm,Minister,Loyola,de,Palacio,had,earlier,accused,Fischler,...,of,causing,unjustified,alarm,through,"""",dangerous,generalisation,.,""""
Taga,B-MISC,O,O,B-PER,I-PER,I-PER,O,O,O,B-PER,...,O,O,O,O,O,O,O,O,O,O


In [ ]:
# let's take a look if there any unusual imbalance in the tags
ar_data = dataset.map(tag_names)
freq = defaultdict(Counter)
for split, data in ar_data.items():
  for row in data["ner_tagss"]:
    for tag in row:
      if tag.startswith("B"):
        tag_type = tag.split("-")[1]
        freq[split][tag_type] += 1
pd.DataFrame.from_dict(freq, orient="index")

,ORG,MISC,PER,LOC
train,6321,3438,6600,7140
validation,1341,922,1842,1837
test,1661,702,1617,1668


In [ ]:
notebook_login()

In [ ]:
xlm_model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(xlm_model_name)

In [ ]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [ ]:
xlm_config = AutoConfig.from_pretrained(xlm_model_name, num_labels=tags.num_classes, id2label= index2tag, label2id=tag2index)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForTokenClassification.from_pretrained(xlm_model_name,
                                                         config = xlm_config).to(device)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [ ]:
text = "everybody know who is messi"
tokens = tokenizer(text).tokens()
input_ids = tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([tokens, input_ids[0].numpy()], index = ["Tokens", "Input ids"])

,0,1,2,3,4,5,6
Tokens,<s>,▁everybody,▁know,▁who,▁is,▁messi,</s>
Input ids,0,195348,3714,2750,83,130823,2


In [ ]:
outputs = model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"numper of tokens {len(tokens)}")
print(f"number of outputs {outputs.shape}")
# the outputs have shape of (batch_size, num_tokens, num_tags)

numper of tokens 7
number of outputs torch.Size([1, 7, 9])


In [ ]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6
Tokens,<s>,▁everybody,▁know,▁who,▁is,▁messi,</s>
Tags,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG,I-ORG


In [ ]:
example_from_dataset = dataset["train"][59]
words,labels = example_from_dataset["tokens"], example_from_dataset["ner_tags"]

tokenized_input = tokenizer(words, is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word_ids"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,<s>,▁--,▁Dimitri,s,▁Konto,gian,nis,▁,",",▁Athen,s,▁News,room,▁+,301,▁33,118,12,-4,</s>
Word_ids,None,0,1,1,2,2,2,3,3,4,4,5,5,6,6,7,7,7,7,None


In [ ]:
def tag_text(text, tags, model, tokenizer):
  tokens = tokenizer(text).tokens()
  input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
  outputs = model(input_ids)[0]
  predictions = torch.argmax(outputs, dim=-1)
  preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
  return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

In [ ]:
def tokenize_and_align_label(examples):

  tokenized_inputs = tokenizer(examples['tokens'], truncation = True, is_split_into_words = True)

  labels = []

  for idx, label in enumerate(examples['ner_tags']):
    word_ids = tokenized_inputs.word_ids(batch_index = idx)
    prev_word_idx = None
    label_ids = []

    for word_idx in word_ids:
      if word_idx is None or word_idx == prev_word_idx:
        label_ids.append(-100)
      elif word_idx != prev_word_idx:
        label_ids.append(label[word_idx])

      prev_word_idx = word_idx

    labels.append(label_ids)

  tokenized_inputs["labels"] = labels

  return tokenized_inputs

In [ ]:
def encode_dataset(dataset):
  return dataset.map(tokenize_and_align_label, batched=True, remove_columns=["id", "tokens", "pos_tags", "ner_tags", "chunk_tags"])

dataset_encoded = encode_dataset(dataset)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [ ]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
dataset_encoded["train"][0]

{'input_ids': [0,
  3747,
  456,
  75161,
  7,
  30839,
  11782,
  47,
  25299,
  47924,
  18,
  56101,
  21,
  6492,
  6,
  5,
  2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100,
  3,
  0,
  -100,
  -100,
  7,
  0,
  0,
  0,
  -100,
  -100,
  7,
  0,
  -100,
  0,
  -100,
  -100]}

In [ ]:
# we alig the predictions for mesuring the metrics
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

In [ ]:
def compute_metrics(eval_pred):
  y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
  return {"f1_score": f1_score(y_true, y_pred)}

In [ ]:
num_epochs = 3
batch_size = 16
learning_rate = 2e-5
logging_steps = len(dataset_encoded["train"]) // batch_size
data_collator = DataCollatorForTokenClassification(tokenizer)
model_name = "ner-xlm-roberta-english"
training_args = TrainingArguments(
    num_train_epochs = num_epochs,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    evaluation_strategy = "epoch",
    logging_steps = logging_steps,
    output_dir = model_name,
    log_level="error",
    save_steps = 1e6,
    weight_decay=0.01,
    disable_tqdm=False,
    fp16 = False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    tokenizer=tokenizer,
    args=training_args)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Score
1,0.162600,0.055434,0.923784
2,0.040400,0.046452,0.941660
3,0.023700,0.044477,0.945223


TrainOutput(global_step=2634, training_loss=0.07554599075898494, metrics={'train_runtime': 715.2634, 'train_samples_per_second': 58.892, 'train_steps_per_second': 3.683, 'total_flos': 1186084706262336.0, 'train_loss': 0.07554599075898494, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(commit_message="Training completed!")

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

events.out.tfevents.1720516663.22ea4ca6b9b9.13761.0:   0%|          | 0.00/7.19k [00:00<?, ?B/s]

events.out.tfevents.1720516028.22ea4ca6b9b9.183.2:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mu7annad/ner-xlm-roberta-english/commit/b3dca2968206a9c710a63950bca5946dadfc6bd6', commit_message='Training completed!', commit_description='', oid='b3dca2968206a9c710a63950bca5946dadfc6bd6', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "everone loves football know who leo messi is"
tag_text(text, tags, trainer.model, tokenizer)

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,<s>,▁ever,one,▁love,s,▁football,▁know,▁who,▁leo,▁messi,▁is,</s>
Tags,O,O,O,O,O,O,O,O,B-PER,I-PER,O,O


In [ ]:
!pip install --upgrade transformers --progress-bar off


  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "Mu7annad/ner-xlm-roberta-english"

text = "My name is Muhannad Ashraf and I live in Cairo"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
classifier = pipeline("ner", model=model, tokenizer = tokenizer, device = device)
output = classifier(text)

output

[{'entity': 'B-PER',
  'score': 0.99719715,
  'index': 4,
  'word': '▁Muha',
  'start': 11,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.98881334,
  'index': 5,
  'word': 'n',
  'start': 15,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99625504,
  'index': 6,
  'word': 'nad',
  'start': 16,
  'end': 19},
 {'entity': 'I-PER',
  'score': 0.99857676,
  'index': 7,
  'word': '▁Ashraf',
  'start': 20,
  'end': 26},
 {'entity': 'B-LOC',
  'score': 0.9976597,
  'index': 12,
  'word': '▁Cair',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9944917,
  'index': 13,
  'word': 'o',
  'start': 45,
  'end': 46}]

In [ ]:
def transform_ner_output(ner_output):
    transformed_output = []
    current_entity = None
    current_words = []
    current_start = None

    for i in range(len(ner_output)):
        item = ner_output[i]
        entity = item['entity'][2:]
        word = item['word']

        if item['entity'].startswith('B-'):
            if current_entity:
                transformed_output.append({
                    'entity': current_entity,
                    'word': ''.join(current_words).replace('▁', ' ').strip(),
                    'start': current_start,
                    'end': ner_output[i - 1]['end']
                })
            current_entity = entity
            current_words = [word]
            current_start = item['start']+1
        elif item['entity'].startswith('I-') and entity == current_entity:
            current_words.append(word)

    if current_entity:
        transformed_output.append({
            'entity': current_entity,
            'word': ''.join(current_words).replace('▁', ' ').strip(),
            'start': current_start+1,
            'end': ner_output[-1]['end']
        })

    return transformed_output

transform_ner_output(output)

[{'entity': 'PER', 'word': 'Muhannad Ashraf', 'start': 11, 'end': 26},
 {'entity': 'LOC', 'word': 'Cairo', 'start': 41, 'end': 46}]